<a href="https://colab.research.google.com/github/Alafiade/Evaluation-of-Regularization-techniques-on-a-CNN-network/blob/main/EVALUATING_MODEL'S_PERFORMANCE_ON_REGULARIZATION_TECHNIQUES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTING LIBRARIES

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
train = datasets.CIFAR10('/data', train = True, download = True, transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))

]))



test = datasets.CIFAR10('/data', train = False, download=True, transform =transforms.ToTensor())

100%|██████████| 170M/170M [00:01<00:00, 104MB/s]


In [ ]:
len(test),(train)

(10000,
 Dataset CIFAR10
     Number of datapoints: 50000
     Root location: /data
     Split: Train
     StandardTransform
 Transform: Compose(
                ToTensor()
                Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
            ))

In [ ]:
train_loader = DataLoader(train, batch_size=64, shuffle=True)
test_loader = DataLoader(test, batch_size=64, shuffle=False)

CNN ARCHITECTURE + BATCHNORM

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MyCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,kernel_size=3 , padding = 1)
    self. conv1_batchnorm = nn.BatchNorm2d(16)
    self.conv2 = nn.Conv2d(16,32, kernel_size=3, padding=1)
    self.conv2_batchnorm = nn.BatchNorm2d(32)
    self.conv3 = nn.Conv2d(32,64,kernel_size=3, padding=1)
    self.conv3_batchnorm = nn.BatchNorm2d(64)
    self.fc1 = nn.Linear(1024, 512)
    self.fc2 = nn.Linear(512,128)
    self.fc3 = nn.Linear(128,10)



  def forward(self,x):
    out = self.conv1_batchnorm(self.conv1(x))
    out = F.max_pool2d(torch.relu(out),2)
    out = self.conv2_batchnorm(self.conv2(out))
    out = F.max_pool2d(torch.relu(out),2)
    out = self.conv3_batchnorm(self.conv3(out))
    out = F.max_pool2d(torch.relu(out),2)
    out = out.view(-1,1024)
    out = torch.relu(self.fc1(out))
    out = torch.relu(self.fc2(out))
    out = self.fc3(out)
    return out



model_batchnorm = MyCNN()










In [ ]:
device = (torch.device('cuda')if torch.cuda.is_available()else torch.device('cpu'))
print(f'Training on device {device}')

Training on device cpu


In [ ]:
model_batchnorm.to(device)

MyCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_batchnorm): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_batchnorm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
from torchsummary import summary

summary(model_batchnorm, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 32, 16, 16]           4,640
       BatchNorm2d-4           [-1, 32, 16, 16]              64
            Conv2d-5             [-1, 64, 8, 8]          18,496
       BatchNorm2d-6             [-1, 64, 8, 8]             128
            Linear-7                  [-1, 512]         524,800
            Linear-8                  [-1, 128]          65,664
            Linear-9                   [-1, 10]           1,290
Total params: 615,562
Trainable params: 615,562
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.44
Params size (MB): 2.35
Estimated Total Size (MB): 2.80
-------------------------------------------

In [ ]:
import torch.optim as optim

In [ ]:
optimizer = optim.SGD(model_batchnorm.parameters(), lr=0.01, momentum=0.9)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
import datetime

def training_loop(n_epochs, optimizer, model_batchnorm, criterion,train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device = device)
      outputs = model_batchnorm(imgs)
      loss = criterion(outputs,labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()

    if epoch ==1 or epoch % 10 == 0:
      print('{}Epoch {}, Training loss {}'.format(
          datetime.datetime.now(), epoch,
          loss_train / len(train_loader)
      ))

In [ ]:
training_loop(
    n_epochs = 50,
    optimizer = optimizer,
    model_batchnorm = model_batchnorm,
    criterion = criterion,
    train_loader = train_loader
)

2026-02-14 16:01:47.288956Epoch 1, Training loss 1.3422732337204086
2026-02-14 16:11:57.499688Epoch 10, Training loss 0.24359249493197713
2026-02-14 16:23:10.307980Epoch 20, Training loss 0.04196545444107364
2026-02-14 16:34:28.636906Epoch 30, Training loss 0.01449987638694168
2026-02-14 16:45:38.079818Epoch 40, Training loss 0.013341634714914322
2026-02-14 16:56:50.554032Epoch 50, Training loss 0.00639724305596865


In [ ]:
def validate(model_batchnorm, train_loader,test_loader):
  model_batchnorm.eval()
  for name, loader in [('train',train_loader),('test',test_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        outputs = model_batchnorm(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print('Accuracy{}: {:.2f}'.format(name, correct/total))

validate(model_batchnorm,train_loader,test_loader)

Accuracytrain: 1.00
Accuracytest: 0.46


CNN + DROPOUT

In [ ]:
class MyCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,kernel_size=3 , padding = 1)
    self.conv1_dropout = nn.Dropout2d(p=0.3)
    self.conv2 = nn.Conv2d(16,32, kernel_size=3, padding=1)
    self.conv2_dropout = nn.Dropout2d(p=0.3)
    self.conv3 = nn.Conv2d(32,64,kernel_size=3, padding=1)
    self.conv3_dropout = nn.Dropout2d(p=0.3)
    self.fc1 = nn.Linear(1024, 512)
    self.fc2 = nn.Linear(512,128)
    self.fc3 = nn.Linear(128,10)



  def forward(self,x):
    out = F.max_pool2d(torch.relu(self.conv1(x)),2)
    out = self.conv1_dropout(out)
    out = F.max_pool2d(torch.relu(self.conv2(out)),2)
    out = self.conv2_dropout(out)
    out = F.max_pool2d(torch.relu(self.conv3(out)),2)
    out = self.conv3_dropout(out)
    out = out.view(-1,1024)
    out = torch.relu(self.fc1(out))
    out = torch.relu(self.fc2(out))
    out = self.fc3(out)
    return out




model_dropout = MyCNN()

In [ ]:
device = (torch.device('cuda')if torch.cuda.is_available()else torch.device('cpu'))
print(f'Training on device {device}')

Training on device cpu


In [ ]:
model_dropout.to(device)

MyCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_dropout): Dropout2d(p=0.3, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_dropout): Dropout2d(p=0.3, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_dropout): Dropout2d(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
from torchsummary import summary

summary(model_dropout, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
         Dropout2d-2           [-1, 16, 16, 16]               0
            Conv2d-3           [-1, 32, 16, 16]           4,640
         Dropout2d-4             [-1, 32, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          18,496
         Dropout2d-6             [-1, 64, 4, 4]               0
            Linear-7                  [-1, 512]         524,800
            Linear-8                  [-1, 128]          65,664
            Linear-9                   [-1, 10]           1,290
Total params: 615,338
Trainable params: 615,338
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.28
Params size (MB): 2.35
Estimated Total Size (MB): 2.64
-------------------------------------------

In [ ]:
optimizer = optim.SGD(model_dropout.parameters(), lr=0.01, momentum=0.9)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
import datetime

def training_loop(n_epochs, optimizer, model_dropout, criterion,train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device = device)
      outputs = model_dropout(imgs)
      loss = criterion(outputs,labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()

    if epoch ==1 or epoch % 10 == 0:
      print('{}Epoch {}, Training loss {}'.format(
          datetime.datetime.now(), epoch,
          loss_train / len(train_loader)
      ))

In [ ]:
training_loop(
    n_epochs = 40,
    optimizer = optimizer,
    model_dropout = model_dropout,
    criterion = criterion,
    train_loader = train_loader
)

2026-02-14 17:10:07.380024Epoch 1, Training loss 2.0566487632444144
2026-02-14 17:18:43.838722Epoch 10, Training loss 1.0390094771714466
2026-02-14 17:28:14.282729Epoch 20, Training loss 0.8245574431422421
2026-02-14 17:37:41.954162Epoch 30, Training loss 0.7009051982551584
2026-02-14 17:47:10.410759Epoch 40, Training loss 0.6286586098887427


In [ ]:
def validate(model_dropout, train_loader,test_loader):
  model_dropout.eval()
  for name, loader in [('train',train_loader),('test',test_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        outputs = model_dropout(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print('Accuracy{}: {:.2f}'.format(name, correct/total))

validate(model_dropout,train_loader,test_loader)

Accuracytrain: 0.92
Accuracytest: 0.52


CNN + DROPOUT+BATCHNORM

In [ ]:
class MyCNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,kernel_size=3 , padding = 1)
    self.batchnorm1 = nn.BatchNorm2d(16)
    self.conv1_dropout = nn.Dropout2d(p=0.3)
    self.conv2 = nn.Conv2d(16,32, kernel_size=3, padding=1)
    self.batchnorm2 = nn.BatchNorm2d(32)
    self.conv2_dropout = nn.Dropout2d(p=0.3)
    self.conv3 = nn.Conv2d(32,64,kernel_size=3, padding=1)
    self.batchnorm3 = nn.BatchNorm2d(64)
    self.conv3_dropout = nn.Dropout2d(p=0.3)
    self.fc1 = nn.Linear(1024, 512)
    self.fc2 = nn.Linear(512,128)
    self.fc3 = nn.Linear(128,10)



  def forward(self,x):
    out = self.conv1(x)
    out = self.batchnorm1(out)
    out = F.relu(out)
    out = F.max_pool2d(out,2)
    out = self.conv1_dropout(out)


    out = self.conv2(out)
    out = self.batchnorm2(out)
    out = F.relu(out)
    out = F.max_pool2d(out,2)
    out = self.conv2_dropout(out)


    out = self.conv3(out)
    out = self.batchnorm3(out)
    out = F.relu(out)
    out = F.max_pool2d(out,2)
    out = self.conv3_dropout(out)


    out = out.view(-1,1024)
    out = F.relu(self.fc1(out))
    out = F.relu(self.fc2(out))
    out = self.fc3(out)

    return out






model_dropnorm = MyCNN()

In [ ]:
model_dropnorm.to(device)

MyCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1_dropout): Dropout2d(p=0.3, inplace=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2_dropout): Dropout2d(p=0.3, inplace=False)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3_dropout): Dropout2d(p=0.3, inplace=False)
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
from torchsummary import summary

summary(model_dropnorm, input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
         Dropout2d-3           [-1, 16, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]           4,640
       BatchNorm2d-5           [-1, 32, 16, 16]              64
         Dropout2d-6             [-1, 32, 8, 8]               0
            Conv2d-7             [-1, 64, 8, 8]          18,496
       BatchNorm2d-8             [-1, 64, 8, 8]             128
         Dropout2d-9             [-1, 64, 4, 4]               0
           Linear-10                  [-1, 512]         524,800
           Linear-11                  [-1, 128]          65,664
           Linear-12                   [-1, 10]           1,290
Total params: 615,562
Trainable params: 615,562
Non-trainable params: 0
-------------------------------

In [ ]:
optimizer = optim.SGD(model_dropnorm.parameters(), lr=0.01, momentum=0.9)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
import datetime

def training_loop(n_epochs, optimizer, model_dropnorm, criterion,train_loader):
  for epoch in range(1, n_epochs +1):
    loss_train = 0.0
    for imgs, labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device = device)
      outputs = model_dropnorm(imgs)
      loss = criterion(outputs,labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()

    if epoch ==1 or epoch % 10 == 0:
      print('{}Epoch {}, Training loss {}'.format(
          datetime.datetime.now(), epoch,
          loss_train / len(train_loader)
      ))

In [ ]:
training_loop(
    n_epochs = 40,
    optimizer = optimizer,
    model_dropnorm = model_dropnorm,
    criterion = criterion,
    train_loader = train_loader
)

2026-02-14 18:16:29.592092Epoch 1, Training loss 1.7541827100622074
2026-02-14 18:26:44.345801Epoch 10, Training loss 1.0122095187911597
2026-02-14 18:38:02.881182Epoch 20, Training loss 0.7941719533308692
2026-02-14 18:49:24.051609Epoch 30, Training loss 0.669480090022392
2026-02-14 19:00:44.013628Epoch 40, Training loss 0.5685908108416116


In [ ]:
def validate(model_dropnorm, train_loader,test_loader):
  model_dropnorm.eval()
  for name, loader in [('train',train_loader),('test',test_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        outputs = model_dropnorm(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
    print('Accuracy{}: {:.2f}'.format(name, correct/total))

validate(model_dropnorm,train_loader,test_loader)

Accuracytrain: 0.93
Accuracytest: 0.54
